In [71]:
from decimal import Decimal

def tax_annual_to_monthly(tax):
    return Decimal(pow(1 + float(tax), 1/12) - 1)


In [72]:
import pandas as pd
import numpy as np


def simulate_financing(parameters):

    orcamento = parameters['orcamento']
    valor_imovel = parameters['valor_imovel']
    aluguel = parameters['aluguel']
    salario = parameters['salario']

    tax_financiamento_anual = parameters['tax_financiamento_anual']
    tax_financiamento_mensal = tax_annual_to_monthly(tax_financiamento_anual)

    prazo_mes = parameters['prazo_mes']
    entrada = parameters['entrada']
    outros_custos = parameters['outros_custos']
    valor_financiado = valor_imovel - entrada
    amortizacao_sac = valor_financiado / prazo_mes

    df = pd.DataFrame({
        'mes': [0],
        'ano': [0],
        'prestacao': [entrada + outros_custos],
        'aluguel': [Decimal('0.00')],
        'juros': [Decimal('0.00')],
        'amortizacao': [Decimal('0.00')],
        'saldo_devedor': [valor_financiado],
        'salario':[Decimal('0.00')],
        'aportes_financiando': [Decimal('0.00')],
        'aportes_alugando': entrada + outros_custos,
        'rendimento_financiando': Decimal('0.00'),
        'investimento_financiando': orcamento - (entrada + outros_custos),
        'valorizacao_financiando': Decimal('0.00'),
        'total_financiando': orcamento - valor_financiado,
        'rendimento_alugando': 0,
        'total_alugando': valor_imovel
    })


    precision = 2
    for i in range(1, 361):

        year = i//12

        if i%12 == 1:
            selic_anual = Decimal(np.random.normal(0.08 - 0.002*year, 0.002 + year*0.0001, 1)[0])
            selic_mensal = tax_annual_to_monthly(selic_anual)
            
            igpm = Decimal(np.random.normal(0.09+0.0005*year, 0.005 + year*0.002, 1)[0])
            valorizacao = Decimal(np.random.normal(0.03 - 0.0001*year, 0.005 + year*0.0005, 1)[0])

        row = {}

        row['ano'] = year
        row['mes'] = i    
        row['amortizacao'] = amortizacao_sac if df.iloc[-1]['saldo_devedor'] > 0 else 0
        row['saldo_devedor'] = df.iloc[-1]['saldo_devedor'] - row['amortizacao']
        row['juros'] = round(df.iloc[-1]['saldo_devedor'] * tax_financiamento_mensal, precision)
        row['prestacao'] = row['amortizacao'] + row['juros']
        row['aluguel'] = round(aluguel * (1 + igpm) ** row['ano'], precision)



        row['salario'] = round(salario * (1 + igpm) ** row['ano'], precision)

        # row['salario'] = max(row['prestacao'], row['aluguel'])

        row['aportes_financiando'] = row['salario'] - row['prestacao']


        if row['prestacao'] / row['salario'] > 0.30:
            print('Prestacao maior que o máximo permitido', i, row['prestacao'], row['salario'])
            return None


        row['rendimento_financiando'] = round(df.iloc[-1]['investimento_financiando'] * selic_mensal, precision)
        row['investimento_financiando'] = df.iloc[-1]['investimento_financiando'] + row['aportes_financiando']  + row['rendimento_financiando']
        row['valorizacao_financiando'] = round(valor_imovel * (1 + valorizacao) ** row['ano'], precision)
        row['total_financiando'] = row['investimento_financiando'] + row['valorizacao_financiando'] - row['saldo_devedor']    

        # row['aportes_alugando'] = row['salario'] - row['aluguel']
        # row['rendimento_alugando'] = round(df.iloc[-1]['total_alugando'] * selic_mensal, precision)
        # row['total_alugando'] = df.iloc[-1]['total_alugando'] + row['rendimento_alugando'] + row['aportes_alugando']

        df = pd.concat([df, pd.DataFrame.from_records([row])], ignore_index=True)


    # display(df.iloc[-1])


    return {
        'patrimonio': round(df.iloc[-1]['total_financiando'], 2),
        'primeira_prestacao': round(df.iloc[1]['prestacao'], 2)
    }

In [73]:
import random


output = pd.DataFrame({
    'patrimonio': []
})


for i in range(200):

    parameters = {
        'orcamento': Decimal('200000'),
        'entrada': 200000 + Decimal(random.randint(0, 4))*5000,
        'aluguel': Decimal(2000),
        'valor_imovel': Decimal(400000),
        'tax_financiamento_anual': Decimal('0.06'),
        'prazo_mes': Decimal(random.randint(1, 30)*12),
        'outros_custos': Decimal('10000'),
        'salario': 7500,
    }

    simulation_result = simulate_financing(parameters) 

    if simulation_result is None:
        continue   

    parameters['patrimonio']= simulation_result['patrimonio']
    parameters['primeira_prestacao']= simulation_result['primeira_prestacao']

    print(parameters)

    output = pd.concat([output, pd.DataFrame.from_records([parameters])], ignore_index=True)

sorted_result = output.sort_values('patrimonio', ascending=False)
display(sorted_result)

Prestacao maior que o máximo permitido 1 2508.163333333333333333333333 7500.00
{'orcamento': Decimal('200000'), 'entrada': Decimal('210000'), 'aluguel': Decimal('2000'), 'valor_imovel': Decimal('400000'), 'tax_financiamento_anual': Decimal('0.06'), 'prazo_mes': Decimal('168'), 'outros_custos': Decimal('10000'), 'salario': 7500, 'patrimonio': Decimal('37022351.74'), 'primeira_prestacao': Decimal('2055.78')}
Prestacao maior que o máximo permitido 1 3751.287777777777777777777778 7500.00
Prestacao maior que o máximo permitido 1 2574.17 7500.00
Prestacao maior que o máximo permitido 1 4626.16 7500.00
{'orcamento': Decimal('200000'), 'entrada': Decimal('205000'), 'aluguel': Decimal('2000'), 'valor_imovel': Decimal('400000'), 'tax_financiamento_anual': Decimal('0.06'), 'prazo_mes': Decimal('360'), 'outros_custos': Decimal('10000'), 'salario': 7500, 'patrimonio': Decimal('20845183.29'), 'primeira_prestacao': Decimal('1490.84')}
{'orcamento': Decimal('200000'), 'entrada': Decimal('205000'), 'al

,patrimonio,orcamento,entrada,aluguel,valor_imovel,tax_financiamento_anual,prazo_mes,outros_custos,salario,primeira_prestacao
53,96225721.27,200000,200000,2000,400000,0.06,288,10000,7500.0,1667.95
56,89310130.94,200000,205000,2000,400000,0.06,336,10000,7500.0,1529.53
92,77504278.36,200000,220000,2000,400000,0.06,180,10000,7500.0,1876.16
61,76361634.78,200000,200000,2000,400000,0.06,264,10000,7500.0,1731.09
88,67878762.01,200000,220000,2000,400000,0.06,300,10000,7500.0,1476.16
...,...,...,...,...,...,...,...,...,...,...
81,18497221.62,200000,200000,2000,400000,0.06,324,10000,7500.0,1590.79
70,18366408.54,200000,205000,2000,400000,0.06,288,10000,7500.0,1626.25
12,18151793.66,200000,200000,2000,400000,0.06,336,10000,7500.0,1568.75
77,17432662.50,200000,215000,2000,400000,0.06,144,10000,7500.0,2185.22
